

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_EN_TREC.ipynb)




# **Classify text according to TREC classes**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.1

In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()

## 3. Select the DL model and re-run all the cells below

In [4]:
### Select Model
#model_name = 'classifierdl_use_trec6'
model_name = 'classifierdl_use_trec50'

## 4. Some sample examples

In [5]:
text_list = [
    "What effect does pollution have on the Chesapeake Bay oysters?",
    "What financial relationships exist between Google and its advertisers?",
    "What financial relationships exist between the Chinese government and the Cuban government?",
    "What was the number of member nations of the U.N. in 2000?",
    "Who is the Secretary-General for political affairs?",
    "When did the construction of stone circles begin in the UK?",
    "In what country is the WTO headquartered?",
    "What animal was the first mammal successfully cloned from adult cells?",
    "What other prince showed his paintings in a two-prince exhibition with Prince Charles in London?",
    "Is there evidence to support the involvement of Garry Kasparov in politics?",
  ]

## 5. Define Spark NLP pipeline

In [6]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(lang="en") \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
          .setInputCols(["document", "sentence_embeddings"]) \
          .setOutputCol("class_")

nlpPipeline = Pipeline(
    stages=[
      documentAssembler, 
      use,
      document_classifier
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_trec50 download started this may take some time.
Approximate size to download 21.2 MB
[OK!]


## 6. Run the pipeline

In [7]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 7. Visualize results

In [8]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+------------------------------------------------------------------------------------------------+------------+
|document                                                                                        |class       |
+------------------------------------------------------------------------------------------------+------------+
|What effect does pollution have on the Chesapeake Bay oysters?                                  | DESC_desc  |
|What financial relationships exist between Google and its advertisers?                          | DESC_desc  |
|What financial relationships exist between the Chinese government and the Cuban government?     | DESC_desc  |
|What was the number of member nations of the U.N. in 2000?                                      | NUM_count  |
|Who is the Secretary-General for political affairs?                                             | HUM_ind    |
|When did the construction of stone circles begin in the UK?                                     | LOC_o